<a href="https://colab.research.google.com/github/park-geun-hyeong/practice_pytorch/blob/main/PPG_data_Binary_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score
from tqdm import tqdm_notebook

In [ ]:
training_subject_list = [2,3,4,5,6,7,8,9,10,11,13,14]
validation_subject_list = [15,16]
test_subject_list = [17]

In [ ]:
class PPG_Dataset(Dataset):
    def __init__(self, subject_list):
        super(PPG_Dataset, self).__init__()

        ppg_list=[]
        for sbj_num in subject_list:
            f_name = 'PPG_data/subject_'+str(sbj_num)+'.npy'
            ppg = np.load(f_name)
            ppg_list.append(ppg)

        self.ppg_merge = np.concatenate(ppg_list, axis=0)
                
    def __getitem__(self, index):
        ppg_sample = self.ppg_merge[index, :7680]
        ppg_tensor = torch.tensor(ppg_sample[np.newaxis, :]).float()
        
        label = self.ppg_merge[index, 7680]
        label_tensor = torch.tensor(label.astype(np.int64))

        return ppg_tensor, label_tensor    
    
    def __len__(self):
        return self.ppg_merge.shape[0]
        

In [ ]:
train_dataset = PPG_Dataset(training_subject_list)
val_dataset = PPG_Dataset(validation_subject_list)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=True)

In [ ]:
class Net(nn.Module):
    def __init__ (self):
        super(Net, self).__init__()

        self.flatten = nn.Flatten()
        
        ##### stage1
        self.stage1_model1 = nn.Sequential(
                nn.Conv1d(1,8, 65, 1, 32),
                nn.MaxPool1d(2,2)
        ) 

        self.stage1_model2 = nn.Sequential(
            nn.Conv1d(1,8,65,2,32)
        )

        self.stage1_model3 = nn.Sequential(
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(p=0.3)
        )

        ###### stage2
        self.stage2_model1 = nn.Sequential(
                nn.Conv1d(16,8, 65, 1, 32),
                nn.MaxPool1d(2,2)
        ) 

        self.stage2_model2 = nn.Sequential(
            nn.Conv1d(16,8,65,2,32)
        )

        self.stage2_model3 = nn.Sequential(
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(p=0.3)
        )

        ##### stage3
        self.stage3_model1 = nn.Sequential(
                nn.Conv1d(16,16, 65, 1, 32),
                nn.MaxPool1d(2,2)
        ) 

        self.stage3_model2 = nn.Sequential(
            nn.Conv1d(16,16,65,2,32)
        )

        self.stage3_model3 = nn.Sequential(
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(p=0.3)
        )

        ##### stage4
        self.stage4_model1 = nn.Sequential(
                nn.Conv1d(32,16, 65, 1, 32),
                nn.MaxPool1d(2,2)
        ) 

        self.stage4_model2 = nn.Sequential(
            nn.Conv1d(32,16,65,2,32)
        )

        self.stage4_model3 = nn.Sequential(
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(p=0.3)
        )

        ##### stage5
        self.stage5_model1 = nn.Sequential(
                nn.Conv1d(32,32, 65, 1, 32),
                nn.MaxPool1d(2,2)
        ) 

        self.stage5_model2 = nn.Sequential(
            nn.Conv1d(32,32,65,2,32)
        )

        self.stage5_model3 = nn.Sequential(
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.3)
        )

        ##### stage6
        self.stage6_model1 = nn.Sequential(
                nn.Conv1d(64,32, 65, 1,32),
                nn.MaxPool1d(2,2)
        ) 

        self.stage6_model2 = nn.Sequential(
            nn.Conv1d(64,32,65,2,32)
        )

        self.stage6_model3 = nn.Sequential(
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.3)
        )

        ##### stage7
        self.stage7_model1 = nn.Sequential(
                nn.Conv1d(64,64, 65, 1, 32),
                nn.MaxPool1d(2,2)
        ) 

        self.stage7_model2 = nn.Sequential(
            nn.Conv1d(64,64,65,2,32)
        )

        self.stage7_model3 = nn.Sequential(
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p=0.3)
        )

        ##### stage8
        self.stage8_model1 = nn.Sequential(
                nn.Conv1d(128,64, 65, 1, 32),
                nn.MaxPool1d(2,2)
        ) 

        self.stage8_model2 = nn.Sequential(
            nn.Conv1d(128,64,65,2,32)
        )

        self.stage8_model3 = nn.Sequential(
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p=0.3)
        )

        ##### fc_layer
        self.fc_layer=nn.Sequential(
            nn.Flatten(),
            nn.Linear(3840,  512),
            nn.ReLU(),

            nn.Linear(512, 64),
            nn.ReLU(),

            nn.Linear(64, 8),
            nn.ReLU(),

            nn.Linear(8,2)

        )

    def forward(self, x):

        x1 = self.stage1_model1(x)
        x2 = self.stage1_model2(x)
        cat = torch.cat([x1,x2],dim=1)
        x = self.stage1_model3(cat)

        x1 = self.stage2_model1(x)
        x2 = self.stage2_model2(x)
        cat = torch.cat([x1,x2],dim=1)
        x = self.stage2_model3(cat)

        x1 = self.stage3_model1(x)
        x2 = self.stage3_model2(x)
        cat = torch.cat([x1,x2],dim=1)
        x = self.stage3_model3(cat)

        x1 = self.stage4_model1(x)
        x2 = self.stage4_model2(x)
        cat = torch.cat([x1,x2],dim=1)
        x = self.stage4_model3(cat)
       
        x1 = self.stage5_model1(x)
        x2 = self.stage5_model2(x)
        cat = torch.cat([x1,x2],dim=1)
        x = self.stage5_model3(cat)

        x1 = self.stage6_model1(x)
        x2 = self.stage6_model2(x)
        cat = torch.cat([x1,x2],dim=1)
        x = self.stage6_model3(cat)

        x1 = self.stage7_model1(x)
        x2 = self.stage7_model2(x)
        cat = torch.cat([x1,x2],dim=1)
        x = self.stage7_model3(cat)

        x1 = self.stage8_model1(x)
        x2 = self.stage8_model2(x)
        cat = torch.cat([x1,x2],dim=1)
        x = self.stage8_model3(cat)

        x = self.flatten(x)

        pred = self.fc_layer(x)

        return pred

In [ ]:
model = Net()

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
lr_sc = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=4, verbose=True)

In [ ]:
base_acc = -1

for epoch in tqdm_notebook(range(50)):
    
    train_loss=[]
    model.train()
    for itr, data in enumerate(train_loader):
        optimizer.zero_grad()
        
        ppg, label = data
        
        pred = model(ppg)
        loss = loss_function(pred, label)
        
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.item())

    val_loss=[]
    val_pred=[]
    val_true=[]

    model.eval()
    with torch.no_grad():
        for itr, data in enumerate(val_loader):
            ppg,label=data
            
            cost = model(ppg)
            
            loss = loss_function(cost, label)
            val_loss.append(loss.item())

            pred_cate = torch.argmax(cost, axis=1)
            
            val_pred.append(np.array(pred_cate))
            val_true.append(np.array(label))
            
        val_pred_cat = np.concatenate(val_pred)
        val_true_cat = np.concatenate(val_true)
        acc = accuracy_score(val_pred_cat, val_true_cat)
        
        lr_sc.step(np.mean(val_loss))

    if acc > base_acc:
        base_acc = acc
        torch.save(model, "model.pth")
        
    print(f"epoch: {epoch+1}, train_loss: {np.mean(train_loss):.4f}, val_loss:{np.mean(val_loss):.4f}, val_acc : {acc:.4f}")

  0%|          | 0/50 [00:00<?, ?it/s]

epoch: 1, train_loss: 0.5939, val_loss:0.6072, val_acc : 0.7049
epoch: 2, train_loss: 0.4373, val_loss:0.8636, val_acc : 0.7049
epoch: 3, train_loss: 0.3130, val_loss:0.7824, val_acc : 0.7049
epoch: 4, train_loss: 0.1967, val_loss:0.5726, val_acc : 0.7869
epoch: 5, train_loss: 0.1767, val_loss:0.5659, val_acc : 0.7131
epoch: 6, train_loss: 0.1480, val_loss:0.6856, val_acc : 0.7459
epoch: 7, train_loss: 0.2152, val_loss:0.6534, val_acc : 0.7869
epoch: 8, train_loss: 0.1510, val_loss:0.4132, val_acc : 0.8033
epoch: 9, train_loss: 0.1175, val_loss:0.3314, val_acc : 0.8443
epoch: 10, train_loss: 0.1214, val_loss:0.2475, val_acc : 0.9098
epoch: 11, train_loss: 0.1842, val_loss:0.9009, val_acc : 0.6066
epoch: 12, train_loss: 0.1376, val_loss:0.3074, val_acc : 0.8361
epoch: 13, train_loss: 0.1065, val_loss:0.4363, val_acc : 0.7951
epoch: 14, train_loss: 0.1005, val_loss:0.7151, val_acc : 0.7131
Epoch    15: reducing learning rate of group 0 to 5.0000e-04.
epoch: 15, train_loss: 0.0796, val_lo

In [ ]:
val_dataset = PPG_Dataset(validation_subject_list)
val_loader = DataLoader(dataset=val_dataset, batch_size=100, shuffle=True)

In [ ]:
model_load = torch.load('model.pth')

pred_list = []
label_list = []
for itr, data in enumerate(val_loader):
    ppg, label = data
    
    pred_test = model_load(ppg)
    pred_category = torch.argmax(pred_test, dim=1)
    
    pred_list = pred_list + list(pred_category)
    label_list = label_list + list(label)
    
accu = np.mean( np.array(pred_list)==np.array(label_list) )
print("Validation accuracy: ", accu)

Validation accuracy:  0.9098360655737705


### Do no modify the below codes ###

In [ ]:
net_load = torch.load("model.pth")

In [ ]:
test_dataset = PPG_Dataset(test_subject_list)
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=True)

In [ ]:
pred_list = []
label_list = []
for itr, data in enumerate(test_loader):
    ppg, label = data
    
    pred_test = net_load(ppg)
    pred_category = torch.argmax(pred_test, dim=1)
    
    pred_list = pred_list + list(pred_category)
    label_list = label_list + list(label)
    
accu = np.mean( np.array(pred_list)==np.array(label_list) )
print("Test accuracy: ", accu)